In [6]:
from agno.models import groq
import os
from dotenv import load_dotenv
import re

load_dotenv()


False

In [7]:
def extract_terdakwa_data(content: str) -> list:
    """
    Mengekstrak Informasi terdakwa dan return sebagai list of dictionaries.
    Setiap dictionary berisi:
        - Nama
        - Tempat Lahir
        - Umur (tanggal lahir)
        - Jenis Kelamin
        - Tempat tinggal
        - Agama
        - Pekerjaan
    """
    terdakwa_pattern = re.compile(
        r"(?:T\s*erdakwa\s*\d*\s*\d*\.\s*)?"
        r"(?:\d+\.\s*)?Nama\s*lengkap\s*:?\s*(?P<nama_lengkap>[^;]+);\s*"
        r"(?:\d+\.\s*)?Tempat\s*lahir\s*:?\s*(?P<tempat_lahir>[^;]+);\s*"
        r"(?:\d+\.\s*)?Umur/Tanggal\s*lahir\s*:?\s*(?P<umur>[^;]+);\s*"
        r"(?:\d+\.\s*)?Jenis\s*kelamin\s*:?\s*(?P<jenis_kelamin>[^;]+);\s*"
        r"(?:\d+\.\s*)?Kebangsaan\s*:?\s*(?P<kebangsaan>[^;]+);\s*"
        r"(?:\d+\.\s*)?Tempat\s*tinggal\s*:?\s*(?P<tempat_tinggal>[^;]+);\s*"
        r"(?:\d+\.\s*)?Agama\s*:?\s*(?P<agama>[^;]+);\s*"
        r"(?:\d+\.\s*)?Pekerjaan\s*:?\s*(?P<pekerjaan>[^;]+)[.;]",
        re.IGNORECASE
    )

    results = []
    for match in terdakwa_pattern.finditer(content):
        data = match.groupdict()
        # Bersihkan nilai dengan strip()
        cleaned_data = {key: value.strip() for key, value in data.items()}
        results.append(cleaned_data)

    return results

In [8]:
def extract_tuntutan_pidana(content: str) -> list:
    """
    Mengekstrak informasi tuntutan pidana dari dokumen.
    """
    # Regex pattern untuk mengekstrak informasi tuntutan pidana
    pattern = re.compile(
        r"(?:T\s*untutan\s*pidana\s*\d*\s*\d*\.\s*)?"
        r"(?:\d+\.\s*)?Tuntutan\s*pidana\s*:?\s*(?P<tuntutan_pidana>[^;]+);",
        re.IGNORECASE
    )

    results = []
    for match in pattern.finditer(content):
        data = match.groupdict()
        # Bersihkan nilai dengan strip()
        cleaned_data = {key: value.strip() for key, value in data.items()}
        results.append(cleaned_data)

    return results

In [76]:
import re

def normalize_pasal(pasal):
    # Hilangkan spasi ekstra, seragamkan ejaan "Jo", "KUHP", dll
    pasal = re.sub(r'\s+', ' ', pasal.lower())
    pasal = pasal.replace('kitab undang - undang hukum pidana', 'kuhp')
    pasal = pasal.replace('kitab undang-undang hukum pidana', 'kuhp')
    pasal = pasal.replace('jo.', 'jo')
    pasal = pasal.replace('junctor', 'jo')
    return pasal.strip()

# def related_pasal(text):
#     pattern = r"""(?xi)
#         pasal\s+\d+(?:\s+ayat\s*\(\d+\))?
#         (?:\s+ke-?\d+)? 
#         (?:\s+jo\.?\s+pasal\s+\d+(?:\s+ayat\s*\(\d+\))?(?:\s+ke-?\d+)?)* 
#         (?:\s+dan\s+ayat\s*\(\d+\))*
#         (?:\s+(?:kuhp|kitab\s+undang[\s-]*undang\s+hukum\s+pidana))?
#     """
#     matches = re.findall(pattern, text, re.MULTILINE)
#     seen = set()
#     for m in matches:
#         norm = normalize_pasal(m)
#         if norm not in seen:
#             seen.add(norm)
#             return m.strip()
#     return None

def related_pasal(text):
    pattern = r'''(?ix)
        pasal\s+\d+                                   # Pasal 378
        (?:\s+ayat\s+\(?\d+\)?                        # ayat (1)
            (?:\s*,\s*ayat\s+\(?\d+\)?)*              # , ayat (2)
            (?:\s*(?:dan|,)\s*ayat\s+\(?\d+\)?)?      # dan ayat (3)
        )?
        (?:\s+ke-?\d+)?                               # ke-1
        (?:\s+UU\s+RI\s+No\.\s*\d+\s*Tahun\s*\d+)?    # UU RI No. 17 Tahun 2023
        (?:\s+(?:KUHP|Kitab\s+Undang[\s-]*Undang\s+Hukum\s+Pidana))?
        (?:
            \s+(?:Jo\.?|Juncto)\s+Pasal\s+\d+         # Jo. Pasal 55
            (?:\s+ayat\s+\(?\d+\)?                    # ayat (1)
                (?:\s*,\s*ayat\s+\(?\d+\)?)*          # , ayat (2)
                (?:\s*(?:dan|,)\s*ayat\s+\(?\d+\)?)?  # dan ayat (3)
            )?
            (?:\s+ke-?\d+)?                           # ke-1
            (?:\s+(?:KUHP|Kitab\s+Undang[\s-]*Undang\s+Hukum\s+Pidana))?
        )*
    '''
    matches = re.findall(pattern, text, re.MULTILINE)
    return list(dict.fromkeys(m.strip() for m in matches if m.strip()))



In [64]:
def get_entity_with_re(file_path: str) -> str:
    """
    Extracts the entity name from a file path using a regular expression.
    The function reads the content of the file, applies a regex pattern to find
    the entity name, and returns the matched entity.
    Args:
    file_path (str): The path of txt file file.

    Returns:
    str: The matched entity name.
    """
    entities = {
        "Nomor Putusan": None,
        "Informasi Terdakwa": None,
        "Tuntutan Pidana": None,
        "Related Pasal": None,
        "Tuntutan Hukuman": None,
        "Putusan Pidana": None,
        "Putusan Hukuman": None,
        "Tanggal Putusan": None,
        "Hakim Ketua": None,
        "Hakim Anggota": None,
        "Penuntut Umum": None,
    }
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        content = file.read()

    pattern = re.compile(
        r"P\s+U\s+T\s+U\s+S\s+A\s+N\s+(?:\n\s*)?Nomor\s+(\d{1,3}/Pid\.(?:B|Sus)/\d{4}/PN\s+\w+)",
        re.MULTILINE
    )
    match = pattern.search(content)
    if match:
        entities["Nomor Putusan"] = match.group(1)
    
    terdakwa_data = extract_terdakwa_data(content)
    if terdakwa_data:
        entities["Informasi Terdakwa"] = terdakwa_data
    

    tuntutan_pidana = extract_tuntutan_pidana(content)
    if tuntutan_pidana:
        entities["Tuntutan Pidana"] = tuntutan_pidana[0]["tuntutan_pidana"]
    # return entities

    related_pasal_matches = related_pasal(content)
    if related_pasal_matches:
        entities["Related Pasal"] = related_pasal_matches
    return entities

In [78]:
# 171.Pid.Sus.2024.PN.Gpr.txt
# final_output_asli.txt
get_entity_with_re("samples/final_output_asli.txt")

{'Nomor Putusan': '141/Pid.B/2022/PN Pbl',
 'Informasi Terdakwa': [{'nama_lengkap': 'Mohammad Novel Al Novel Bin Edi Kasiono Alm.',
   'tempat_lahir': 'Probolinggo',
   'umur': '19 tahun/12 Agustus 2003',
   'jenis_kelamin': 'Laki-laki',
   'kebangsaan': 'Indonesia',
   'tempat_tinggal': 'Jl. Pahlawan Gg. VII Kel. Kebonsari Kulon Kec. Kanigaran, Kota Probolinggo',
   'agama': 'Islam',
   'pekerjaan': 'Belum bekerja. Terdakwa Mohammad Novel Al Novel Bin Edi Kasiono Alm . ditangkap pada tanggal 29 September 2022. Terdakwa Mohammad Novel Al Novel Bin Edi Kasiono Alm ditahan dalam tahanan rutan oleh:  1. Penyidik sejak tanggal 30 September 2022 sampai dengan tanggal 19 Oktober 2022'},
  {'nama_lengkap': 'Deva Bagus Prasetyo Al Deva',
   'tempat_lahir': 'Probolinggo',
   'umur': '21 tahun/6 Agustus 2001',
   'jenis_kelamin': 'Laki-laki',
   'kebangsaan': 'Indonesia',
   'tempat_tinggal': 'Jl. KH. Hasan Genggong, Gg Pendekar, Kel. Sukoharjo, Kec. Kanigaran Kota Probolinggo',
   'agama': 'Isl

In [ ]:

def get_entity_with_groq(file_path: str) -> str:
    """
    Extracts the entity name from a file path using groq.
    The function reads the content of the file, applies a groq pattern to find
    the entity name, and returns the matched entity.
    Args:
    file_path (str): The path of txt file file.

    Returns:
    str: The matched entity name.
    """
    entities = groq(file_path)
    return entities

get_entity_with_re("txts/final/171.Pid.Sus.2024.PN.Gpr.txt")
get_entity_with_groq("txts/final/171.Pid.Sus.2024.PN.Gpr.txt")